In [1]:
using DifferentialEquations
include("src/Utils.jl")
include("src/Models.jl")
include("src/Params.jl");
using Plots; pythonplot();
using Statistics
using Contour

In [2]:
"""
At low temperatures kSU can be different from kSUB. See Axmann et al. 2004 Fig. 4 and Gopal's preliminary data
"""
function kaiabc_phong_low_temp3!(dX, X, p, t)

    # U: unphosphorylated. T: only T432 phosphorylated. S: only S431 phosphorylated
    # D: doubly phosphorylated. DB: D KaiC bound with KaiB. SB: S KaiC bound with KaiB
    U, T, D, S, DB, SB = X

    # k's are reaction constants (here all are 1st-order), e.g., 
    # kTU is the per reactant rate transfering from T to U
    # kCIhyd: ATP hydrolysis rate of the CI ring
    # KA: [KaiA] that activates kinase activity to half maximum
    # A0: initial [KaiA]
    # fATP = [ATP] / ([ATP] + [ADP])
    # N: # A sequestered per D. M: # A sequestered per S
    # note that this model does not include K_ASB
    kUTA, kTU, kTUA, kTDA, kDT, kDTA, kDS, kDSA, kSDA, kSU, kSUA, kUSA, kCIhyd,
    KA, A0, fATP, N, M, K_ASB, κ, ν = p

    hill(x) = x/(x + KA)

    # More realistic KaiA binding with a Kd
    # !!Ignored sequestration of KaiA by DB
    A = A0*(κ^ν/(κ^ν + (SB)^ν))    # free KaiA
    
    #           U,      T,      D,      S,      DB,     SB
    dephos_r  = [0      kTU     0       kSU     0       kSU
                 0      0       kDT     0       kDT     0
                 0      0       0       0       0       0
                 0      0       kDS     0       0       0
                 0      0       0       0       0       0
                 0      0       0       0       kDS     0] +
                [0      kTUA    0       kSUA    0       kSUA
                 0      0       kDTA    0       kDTA    0
                 0      0       0       0       0       0
                 0      0       kDSA    0       0       0
                 0      0       0       0       0       0
                 0      0       0       0       kDSA    0]*hill(A)
    
    #         U     T       D       S       DB      SB
    phos_r = [0     0       0       0       0       0
              kUTA  0       0       0       0       0
              0     kTDA    0       kSDA    0       0
              kUSA  0       0       0       0       0
              0     0       0       0       0       kSDA
              0     0       0       0       0       0]*hill(A)*fATP

    #        U      T       D       S       DB      SB
    hyd_r = [0      0       0       0       0       0
             0      0       0       0       0       0
             0      0       0       0       0       0
             0      0       0       0       0       0
             0      0       kCIhyd  0       0       0
             0      0       0       kCIhyd  0       0]

    r = dephos_r + phos_r + hyd_r

    # conservation
    r -= I(size(r)[1]) .* sum(r, dims=1)

    # (for AD) Zygote.Buffer doesn't support in-place broadcast .=
    # see https://discourse.julialang.org/t/how-to-use-initialize-zygote-buffer/87653
    dX[:] = r*X
    nothing
end

kaiabc_phong_low_temp3!

In [4]:
κ = 0.06
ν = 9.

fATP = 1.
A0 = 1.

M_new = 6.
K_ASB = 0.1

u0 = [3.5, 0, 0, 0, 0, 0]
        
        p = (kUTA, kTU, kTUA, kTDA, kDT, kDTA, kDS, kDSA, kSDA, 0.03, kSUA, kUSA, kCIhyd,
        KA, A0, fATP, 0., M_new, K_ASB, κ, ν)
        
        # solve it one more time to get mean fraction of pC
        prob = ODEProblem(kaiabc_phong_low_temp3!, u0, (0.0, 200), p)
        sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)

conv_t, amplitude, per, phase = 
            attracted_to(kaiabc_phong_low_temp3!, u0, 168, p; 
            burnin=96, conv_tol=5e-2, amp_cutoff=5e-2, tmax_stop=1e3)

(0.016800000000000002, -1, -1, -1)